In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/infer_data/test.parquet')
df.ts = (df.ts//1000).astype('int32')

df = df.loc[df['type']!=0]

df['hour'] = df.ts % (60*60*24)
df['day'] = df.ts % (60*60*24*7)
print( df.shape )
df.head()

(635491, 6)


,session,aid,ts,type,hour,day
12,12899781,199008,1661781409,1,50209,395809
21,12899782,1494780,1661724244,1,79444,338644
28,12899782,413962,1661765608,1,34408,380008
33,12899782,779477,1661766162,1,34962,380562
35,12899782,562753,1661766178,1,34978,380578


In [3]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique','hour':'mean','day':'mean'})
item_features.columns = ['buy_ratio','count_item','count_user','hour_mean','day_mean']

In [4]:
item_features.head()

,buy_ratio,count_item,count_user,hour_mean,day_mean
aid,,,,,
3,1.136364,22,17,48736.136364,484663.409091
21,1.000000,2,1,57269.500000,316469.500000
24,1.000000,1,1,59457.000000,145857.000000
25,1.000000,1,1,59396.000000,59396.000000
39,1.000000,2,1,27360.000000,27360.000000


In [5]:
item_features2 = df.groupby('aid').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std','hour_std','day_std']

f32 = ['buy_ratio_std','hour_std','day_std']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [6]:
item_features2.head()

,buy_ratio_std,hour_std,day_std
aid,,,
1186591,0.000000,19154.109375,164127.968750
991723,-1.000000,-1.000000,-1.000000
1605154,0.000000,19347.121094,36866.339844
1622086,0.707107,164.048767,164.048767
885354,0.447214,29638.162109,50245.656250


In [7]:
item_features['repeat'] = item_features.count_item / item_features.count_user

f32 = ['buy_ratio','hour_mean','day_mean','repeat']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item','count_user']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [8]:
item_features = cudf.concat([item_features,item_features2],axis=1)

In [9]:
item_features.head()

,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std
aid,,,,,,,,,
3,1.136364,22,17,48736.136719,484663.40625,1.294118,0.35125,17377.468750,74483.609375
21,1.000000,2,1,57269.500000,316469.50000,2.000000,0.00000,47.376156,47.376156
24,1.000000,1,1,59457.000000,145857.00000,1.000000,-1.00000,-1.000000,-1.000000
25,1.000000,1,1,59396.000000,59396.00000,1.000000,-1.00000,-1.000000,-1.000000
39,1.000000,2,1,27360.000000,27360.00000,2.000000,0.00000,4.242640,4.242640


In [10]:
item_features.dtypes

buy_ratio        float32
count_item         int32
count_user         int32
hour_mean        float32
day_mean         float32
repeat           float32
buy_ratio_std    float32
hour_std         float32
day_std          float32
dtype: object

In [11]:
COLS = [f'{x}12' for x in item_features.columns]
item_features.columns = COLS
print(COLS)

['buy_ratio12', 'count_item12', 'count_user12', 'hour_mean12', 'day_mean12', 'repeat12', 'buy_ratio_std12', 'hour_std12', 'day_std12']


In [12]:
item_features.to_parquet('../../data/item_user_features/item41_LB.pqt')